In [14]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import  plot_tree
import matplotlib.pyplot as plt


In [15]:
from sklearn.calibration import LabelEncoder


crab = pd.read_csv("train.csv",index_col=0)
lbl = LabelEncoder()
crab = pd.get_dummies(crab,drop_first=True)
X = crab.drop('Age',axis=1)
y = crab[['Age']]
kfold =KFold(n_splits=5, shuffle=True, random_state=23)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)


In [16]:
from sklearn.tree import DecisionTreeRegressor


dtr = DecisionTreeRegressor(random_state=23, max_depth=None)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=23)

In [17]:

y_pred = dtr.predict(X_test)


Creating a graphical view of tree

In [18]:
# plt.figure(figsize=(15,10))
# plot_tree(dtr, feature_names=X_train.columns, filled=True, fontsize=12)
# plt.show()

In [19]:
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
params = {'max_depth':[2,3,4,5,6,7,8,9,10,None]}
gcv = GridSearchCV(dtr, param_grid=params, cv=kfold, n_jobs=4)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': 8}
0.5555195001857125


In [13]:
best_model = gcv.best_estimator_

df_imp = pd.DataFrame({'variable':best_model.feature_names_in_, 'importance':best_model.feature_importances_})
df_imp = df_imp[df_imp['importance']>0.0001]
df_imp.sort_values(by='importance',ascending=False)
plt.barh(df_imp['variable'],df_imp['importance'])
plt.ylabel('Importance')
plt.show()

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [11]:
params = {'max_depth':[2,3,4,5,6,7,8,9,10,None],'min_samples_split':[2,3,4,5,6,7,8,9,10],'min_samples_leaf':np.arange(1,16)}
gcv = GridSearchCV(dtr, param_grid=params, cv=kfold, n_jobs=4)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

PicklingError: Could not pickle the task to send it to the workers.

In [22]:
test = pd.read_csv('test.csv')
test = pd.get_dummies(test,drop_first=True)
bm = gcv.best_estimator_
y_pred = bm.predict(test.iloc[:,1:])
print(y_pred)
submit = pd.DataFrame({'id':test.id,'Age':y_pred})
submit.to_csv('submission_for_Crab.csv',index=False)

[ 7.9787234   7.71933962 10.22368421 ... 13.40625    10.22368421
 12.24096386]
